In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from scipy.io import wavfile
import os
import sys
import sklearn
import scipy
import librosa
import keras

import numpy as np
from tqdm import tqdm

Using TensorFlow backend.


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
device

device(type='cpu')

In [4]:
input_size = 50
hidden_size = 20
num_layers = 4
num_classes = 3
batch_size = 20
num_epochs = 10
learning_rate = 0.01

In [11]:
class RNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(RNN, self).__init__()
        # Hidden dimensions
        self.hidden_dim = hidden_dim
        
        # Number of hidden layers
        self.layer_dim = layer_dim
        
        # Building your LSTM
        # batch_first=True causes input/output tensors to be of shape
        # (batch_dim, seq_dim, feature_dim)
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)
        
        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
        
        
    def forward(self, input, hidden):
        # Initialize hidden state with zeros
        #######################
        #  USE GPU FOR MODEL  #
        #######################
        if torch.cuda.is_available():
            h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).cuda())
        else:
            h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)) 
       
        # Initialize cell state
        if torch.cuda.is_available():
            c0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).cuda())
        else:
            
            c0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))
        
        # Forward propagate RNN
        out, _ = self.lstm(x, (h0, c0))  
        
        # print (out.size())
        # Decode hidden state of last time step
        output = self.fc(out[:, -1, :])  
        
        return output
    

In [12]:
class LSTMCell(nn.Module):

    """A basic LSTM cell."""

    def __init__(self, input_size, hidden_size, use_bias=True):
        """
        Most parts are copied from torch.nn.LSTMCell.
        """

        super(LSTMCell, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.use_bias = use_bias
        self.weight_ih = nn.Parameter(
            torch.FloatTensor(input_size, 4 * hidden_size))
        self.weight_hh = nn.Parameter(
            torch.FloatTensor(hidden_size, 4 * hidden_size))
        if use_bias:
            self.bias = nn.Parameter(torch.FloatTensor(4 * hidden_size))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        """
        Initialize parameters following the way proposed in the paper.
        """

        init.orthogonal(self.weight_ih.data)
        weight_hh_data = torch.eye(self.hidden_size)
        weight_hh_data = weight_hh_data.repeat(1, 4)
        self.weight_hh.data.set_(weight_hh_data)
        # The bias is just set to zero vectors.
        if self.use_bias:
            init.constant(self.bias.data, val=0)

    def forward(self, input_, hx):
        """
        Args:
            input_: A (batch, input_size) tensor containing input
                features.
            hx: A tuple (h_0, c_0), which contains the initial hidden
                and cell state, where the size of both states is
                (batch, hidden_size).
        Returns:
            h_1, c_1: Tensors containing the next hidden and cell state.
        """

        h_0, c_0 = hx
        batch_size = h_0.size(0)
        bias_batch = (self.bias.unsqueeze(0)
                      .expand(batch_size, *self.bias.size()))
        wh_b = torch.addmm(bias_batch, h_0, self.weight_hh)
        wi = torch.mm(input_, self.weight_ih)
        f, i, o, g = torch.split(wh_b + wi,
                                 split_size=self.hidden_size, dim=1)
        c_1 = torch.sigmoid(f)*c_0 + torch.sigmoid(i)*torch.tanh(g)
        h_1 = torch.sigmoid(o) * torch.tanh(c_1)
        return h_1, c_1

    def __repr__(self):
        s = '{name}({input_size}, {hidden_size})'
        return s.format(name=self.__class__.__name__, **self.__dict__)



In [13]:
class BNLSTMCell(nn.Module):

    """A BN-LSTM cell."""

    def __init__(self, input_size, hidden_size, max_length, use_bias=True):

        super(BNLSTMCell, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.max_length = max_length
        self.use_bias = use_bias
        self.weight_ih = nn.Parameter(
            torch.FloatTensor(input_size, 4 * hidden_size))
        self.weight_hh = nn.Parameter(
            torch.FloatTensor(hidden_size, 4 * hidden_size))
        if use_bias:
            self.bias = nn.Parameter(torch.FloatTensor(4 * hidden_size))
        else:
            self.register_parameter('bias', None)
        # BN parameters
        self.bn_ih = SeparatedBatchNorm1d(
            num_features=4 * hidden_size, max_length=max_length)
        self.bn_hh = SeparatedBatchNorm1d(
            num_features=4 * hidden_size, max_length=max_length)
        self.bn_c = SeparatedBatchNorm1d(
            num_features=hidden_size, max_length=max_length)
        self.reset_parameters()

    def reset_parameters(self):
        """
        Initialize parameters following the way proposed in the paper.
        """

        # The input-to-hidden weight matrix is initialized orthogonally.
        init.orthogonal(self.weight_ih.data)
        # The hidden-to-hidden weight matrix is initialized as an identity
        # matrix.
        weight_hh_data = torch.eye(self.hidden_size)
        weight_hh_data = weight_hh_data.repeat(1, 4)
        self.weight_hh.data.set_(weight_hh_data)
        # The bias is just set to zero vectors.
        init.constant(self.bias.data, val=0)
        # Initialization of BN parameters.
        self.bn_ih.reset_parameters()
        self.bn_hh.reset_parameters()
        self.bn_c.reset_parameters()
        self.bn_ih.bias.data.fill_(0)
        self.bn_hh.bias.data.fill_(0)
        self.bn_ih.weight.data.fill_(0.1)
        self.bn_hh.weight.data.fill_(0.1)
        self.bn_c.weight.data.fill_(0.1)

    def forward(self, input_, hx, time):
        """
        Args:
            input_: A (batch, input_size) tensor containing input
                features.
            hx: A tuple (h_0, c_0), which contains the initial hidden
                and cell state, where the size of both states is
                (batch, hidden_size).
            time: The current timestep value, which is used to
                get appropriate running statistics.
        Returns:
            h_1, c_1: Tensors containing the next hidden and cell state.
        """

        h_0, c_0 = hx
        batch_size = h_0.size(0)
        bias_batch = (self.bias.unsqueeze(0)
                      .expand(batch_size, *self.bias.size()))
        wh = torch.mm(h_0, self.weight_hh)
        wi = torch.mm(input_, self.weight_ih)
        bn_wh = self.bn_hh(wh, time=time)
        bn_wi = self.bn_ih(wi, time=time)
        f, i, o, g = torch.split(bn_wh + bn_wi + bias_batch,
                                 split_size=self.hidden_size, dim=1)
        c_1 = torch.sigmoid(f)*c_0 + torch.sigmoid(i)*torch.tanh(g)
        h_1 = torch.sigmoid(o) * torch.tanh(self.bn_c(c_1, time=time))
        return h_1, c_1


In [14]:
class LSTM(nn.Module):

    """A module that runs multiple steps of LSTM."""

    def __init__(self, cell_class, input_size, hidden_size, num_layers=1,
                 use_bias=True, batch_first=False, dropout=0, **kwargs):
        super(LSTM, self).__init__()
        self.cell_class = cell_class
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.use_bias = use_bias
        self.batch_first = batch_first
        self.dropout = dropout

        for layer in range(num_layers):
            layer_input_size = input_size if layer == 0 else hidden_size
            cell = cell_class(input_size=layer_input_size,
                              hidden_size=hidden_size,
                              **kwargs)
            setattr(self, 'cell_{}'.format(layer), cell)
        self.dropout_layer = nn.Dropout(dropout)
        self.reset_parameters()

    def get_cell(self, layer):
        return getattr(self, 'cell_{}'.format(layer))

    def reset_parameters(self):
        for layer in range(self.num_layers):
            cell = self.get_cell(layer)
            cell.reset_parameters()

    @staticmethod
    def _forward_rnn(cell, input_, length, hx):
        max_time = input_.size(0)
        output = []
        for time in range(max_time):
            if isinstance(cell, BNLSTMCell):
                h_next, c_next = cell(input_=input_[time], hx=hx, time=time)
            else:
                h_next, c_next = cell(input_=input_[time], hx=hx)
            mask = (time < length).float().unsqueeze(1).expand_as(h_next)
            h_next = h_next*mask + hx[0]*(1 - mask)
            c_next = c_next*mask + hx[1]*(1 - mask)
            hx_next = (h_next, c_next)
            output.append(h_next)
            hx = hx_next
        output = torch.stack(output, 0)
        return output, hx

    def forward(self, input_, length=None, hx=None):
        if self.batch_first:
            input_ = input_.transpose(0, 1)
        max_time, batch_size, _ = input_.size()
        if length is None:
            length = Variable(torch.LongTensor([max_time] * batch_size))
            if input_.is_cuda:
                device = input_.get_device()
                length = length.cuda(device)
        if hx is None:
            hx = Variable(input_.data.new(batch_size, self.hidden_size).zero_())
            hx = (hx, hx)
        h_n = []
        c_n = []
        layer_output = None
        for layer in range(self.num_layers):
            cell = self.get_cell(layer)
            layer_output, (layer_h_n, layer_c_n) = LSTM._forward_rnn(
                cell=cell, input_=input_, length=length, hx=hx)
            input_ = self.dropout_layer(layer_output)
            h_n.append(layer_h_n)
            c_n.append(layer_c_n)
        output = layer_output
        h_n = torch.stack(h_n, 0)
        c_n = torch.stack(c_n, 0)
        return output, (h_n, c_n)

## Autoencoder

https://blog.paperspace.com/adversarial-autoencoders-with-pytorch/##

### Encoder
    class Q_net(nn.Module):
        def __init__(self, X_dim = 100, N = 1000, z_dim = 100):
            super(Q_net, self).__init__()
            self.X_dim = X_dim
            self.z_dim = z_dim
            self.N = N
            self.lin1 = nn.Linear(self.X_dim, self.N)
            self.lin2 = nn.Linear(self.N, self.N)
            self.lin3gauss = nn.Linear(self.N, self.z_dim)
        def forward(self, x):
            x = F.droppout(self.lin1(x), p=0.25, training=self.training)
            x = F.relu(x)
            x = F.droppout(self.lin2(x), p=0.25, training=self.training)
            x = F.relu(x)
            xgauss = self.lin3gauss(x)
            return xgauss
            
### Decoder
    class P_net(nn.Module):
        def __init__(self, z_dim = 100, N = 1000):
            super(P_net, self).__init__()
            self.z_dim = z_dim
            self.N = N
            self.lin1 = nn.Linear(self.z_dim, self.N)
            self.lin2 = nn.Linear(self.N, self.N)
            self.lin3 = nn.Linear(self.N, self.X_dim)
        def forward(self, x):
            x = self.lin1(self.x)
            x = F.dropout(x, p=0.25, training=self.training)
            x = F.relu(x)
            x = self.lin2(x)
            x = F.dropout(x, p=0.25, training=self.training)
            x = self.lin3(x)
            return F.sigmoid(x)
            
### Discriminator
    class D_net_gauss(nn.Module):
        def __init__(self, z_dim = 100, N = 1000):
            super(D_net_gauss, self).__init__()
            self.z_dim = z_dim
            self.N = N
            self.lin1 = nn.Linear(self.z_dim, self.N)
            self.lin2 = nn.Linear(self.N, self.N)
            self.lin3 = nn.Linear(self.N, 1)
        def forward(self, x):
            x = F.dropout(self.lin1(x), p=0.2, training=self.training)
            x = F.relu(x)
            x = F.dropout(self.lin2(x), p=0.2, training=self.training)
            x = F.relu(x)
            return F.sigmoid(self.lin3(x))

    torch.manual_seed(10)
    Q, P = Q_net(), P_net(0)     # Encoder/Decoder
    D_gauss = D_net_gauss()                # Discriminator adversarial
    if torch.cuda.is_available():
        Q = Q.cuda()
        P = P.cuda()
        D_cat = D_gauss.cuda()
        D_gauss = D_net_gauss().cuda()
   ### Set learning rates
    gen_lr, reg_lr = 0.0006, 0.0008
   ### Set optimizators
    P_decoder = optim.Adam(P.parameters(), lr=gen_lr)
    Q_encoder = optim.Adam(Q.parameters(), lr=gen_lr)
    Q_generator = optim.Adam(Q.parameters(), lr=reg_lr)
    D_gauss_solver = optim.Adam(D_gauss.parameters(), lr=reg_lr)

    z_sample = Q(X)
    X_sample = P(z_sample)
    recon_loss = F.binary_cross_entropy(X_sample + TINY, 
                                            X.resize(train_batch_size, X_dim) + TINY)
    recon_loss.backward()
    P_decoder.step()
    Q_encoder.step()
    Q.eval()    
    z_real_gauss = Variable(torch.randn(train_batch_size, z_dim) * 5)   # Sample from N(0,5)
    if torch.cuda.is_available():
        z_real_gauss = z_real_gauss.cuda()
    z_fake_gauss = Q(X)
    
   ### Compute discriminator outputs and loss
    D_real_gauss, D_fake_gauss = D_gauss(z_real_gauss), D_gauss(z_fake_gauss)
    D_loss_gauss = -torch.mean(torch.log(D_real_gauss + TINY) + torch.log(1 - D_fake_gauss + TINY))
    D_loss.backward()       # Backpropagate loss
    D_gauss_solver.step()   # Apply optimization step
    
   ### Generator
    Q.train()   # Back to use dropout
    z_fake_gauss = Q(X)
    D_fake_gauss = D_gauss(z_fake_gauss)

    G_loss = -torch.mean(torch.log(D_fake_gauss + TINY))
    G_loss.backward()
    Q_generator.step()

### Code for running rnn pytorch

In [97]:
rnn = RNN(input_size, hidden_size, num_layers, num_classes)

NameError: name 'hidden_size' is not defined

In [57]:
if torch.cuda.is_available():
    rnn.cuda()

In [58]:
criterion = nn.CrossEntropyLoss()


optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)

In [59]:
### Get data and set up x

seq_dim = 20
iter = 0


In [60]:
for epoch in range(num_epochs):
    for i, (audio, labels) in enumerate(train_loader):
        # Load images as Variable
        #######################
        #  USE GPU FOR MODEL  #
        #######################
        if torch.cuda.is_available():
            images = Variable(audio.view(-1, seq_dim, input_dim).cuda())
            labels = Variable(labels.cuda())
        else:
            images = Variable(audio.view(-1, seq_dim, input_dim))
            labels = Variable(labels)
            
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
        
        # Forward pass to get output/logits
        # outputs.size() --> 100, 10
        outputs = model(audio)
        
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)
        
        # Getting gradients w.r.t. parameters
        loss.backward()
        
        # Updating parameters
        optimizer.step()
        
        iter += 1
        
        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                #######################
                #  USE GPU FOR MODEL  #
                #######################
                if torch.cuda.is_available():
                    images = Variable(audio.view(-1, seq_dim, input_dim).cuda())
                else:
                    images = Variable(audio.view(-1, seq_dim, input_dim))
                
                # Forward pass only to get logits/output
                outputs = model(audio)
                
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                
                # Total number of labels
                total += labels.size(0)
                
                # Total correct predictions
                #######################
                #  USE GPU FOR MODEL  #
                #######################
                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum()
                else:
                    correct += (predicted == labels).sum()
            
            accuracy = 100 * correct / total
            
            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.data[0], accuracy))

ValueError: too many values to unpack (expected 2)

In [50]:
import logging
import os
import subprocess

import pandas as pd
import numpy as np

from sklearn.manifold import TSNE

def setup_log(tag = 'VOC_TOPICS'):
    global logger
    # create logger
    logger = logging.getLogger(tag)
    # logger.handlers = []
    logger.propagate = False
    logger.setLevel(logging.DEBUG)
    # create console handler and set level to debug
    ch = logging.StreamHandler()
    ch.setLevel(logging.DEBUG)
    # create formatter
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    # add formatter to ch
    ch.setFormatter(formatter)
    # add ch to logger
    # logger.handlers = []
    logger.addHandler(ch)

def setup_path(s3_prefix, data_dir):
    global local_data_dir, s3_input_prefix, s3_bucket
    s3_bucket = 's3://xxxxxx'
    local_data_dir = data_dir
    s3_input_prefix = s3_prefix

def exists_in_s3(key):
    ls = list_in_s3(key)
    if len(ls) > 0:
        if ls[0] == key[(key.rfind('/') + 1):]:
            return True
    return False

def list_in_s3(prefix):
    command = "aws s3 ls s3://{}/{}".format(s3_bucket, prefix)
    ls = subprocess.Popen(command.split(' '), stdout = subprocess.PIPE).stdout.read().split('\n')
    if len(ls) > 0:
        ls = [x[(x.rfind(' ') + 1):] for x in ls]
        return ls
    else:
        return []

def download_file(key):
    ## download
    command = 'aws s3 cp s3://{}/{} {}/'.format(s3_bucket, key, local_data_dir)
    status = subprocess.call(command.split(' '))
    if status != 0:
        logger.error('Error in downloading %s', key)
    else:
        logger.info('Downloaded file %s to local.', key)
        import re
    return

def plot_tsne(data, perplexity = 30, n_iter = 200):
    tsne = TSNE(perplexity = perplexity, n_components = 2, init = 'pca', n_iter = n_iter)
    low_dim_embs = tsne.fit_transform(data)
    plt.scatter(low_dim_embs[:, 0], low_dim_embs[:, 1])

In [53]:
import torch
from torch import nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F

import matplotlib
# matplotlib.use('Agg')
%matplotlib inline

import datetime as dt, itertools, pandas as pd, matplotlib.pyplot as plt, numpy as np




use_cuda = torch.cuda.is_available()
logger.info("Is CUDA available? %s.", use_cuda)


2018-05-22 08:37:26,299 - VOC_TOPICS - INFO - Is CUDA available? False.


In [54]:
class encoder(nn.Module):
    def __init__(self, input_size, hidden_size, T, logger):
        # input size: number of underlying factors (81)
        # T: number of time steps (10)
        # hidden_size: dimension of the hidden state
        super(encoder, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.T = T

        self.logger = logger

        self.lstm_layer = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = 1)
        self.attn_linear = nn.Linear(in_features = 2 * hidden_size + T - 1, out_features = 1)

    def forward(self, input_data):
        # input_data: batch_size * T - 1 * input_size        
        input_weighted = Variable(input_data.data.new(input_data.size(0), self.T - 1, self.input_size).zero_())
        input_encoded = Variable(input_data.data.new(input_data.size(0), self.T - 1, self.hidden_size).zero_())
        # hidden, cell: initial states with dimention hidden_size
        hidden = self.init_hidden(input_data) # 1 * batch_size * hidden_size
        cell = self.init_hidden(input_data)
        # hidden.requires_grad = False
        # cell.requires_grad = False
        for t in range(self.T - 1):
            # Eqn. 8: concatenate the hidden states with each predictor
            x = torch.cat((hidden.repeat(self.input_size, 1, 1).permute(1, 0, 2),
                           cell.repeat(self.input_size, 1, 1).permute(1, 0, 2),
                           input_data.permute(0, 2, 1)), dim = 2) # batch_size * input_size * (2*hidden_size + T - 1)
            # Eqn. 9: Get attention weights
            x = self.attn_linear(x.view(-1, self.hidden_size * 2 + self.T - 1)) # (batch_size * input_size) * 1
            attn_weights = F.softmax(x.view(-1, self.input_size)) # batch_size * input_size, attn weights with values sum up to 1.
            # Eqn. 10: LSTM
            weighted_input = torch.mul(attn_weights, input_data[:, t, :]) # batch_size * input_size
            # Fix the warning about non-contiguous memory
            # see https://discuss.pytorch.org/t/dataparallel-issue-with-flatten-parameter/8282
            self.lstm_layer.flatten_parameters()
            _, lstm_states = self.lstm_layer(weighted_input.unsqueeze(0), (hidden, cell))
            hidden = lstm_states[0]
            cell = lstm_states[1]
            # Save output
            input_weighted[:, t, :] = weighted_input
            input_encoded[:, t, :] = hidden
        return input_weighted, input_encoded

    def init_hidden(self, x):
        # No matter whether CUDA is used, the returned variable will have the same type as x.
        return Variable(x.data.new(1, x.size(0), self.hidden_size).zero_()) # dimension 0 is the batch dimension

class decoder(nn.Module):
    def __init__(self, encoder_hidden_size, decoder_hidden_size, T, logger):
        super(decoder, self).__init__()

        self.T = T
        self.encoder_hidden_size = encoder_hidden_size
        self.decoder_hidden_size = decoder_hidden_size

        self.logger = logger

        self.attn_layer = nn.Sequential(nn.Linear(2 * decoder_hidden_size + encoder_hidden_size, encoder_hidden_size),
                                         nn.Tanh(), nn.Linear(encoder_hidden_size, 1))
        self.lstm_layer = nn.LSTM(input_size = 1, hidden_size = decoder_hidden_size)
        self.fc = nn.Linear(encoder_hidden_size + 1, 1)
        self.fc_final = nn.Linear(decoder_hidden_size + encoder_hidden_size, 1)

        self.fc.weight.data.normal_()

    def forward(self, input_encoded, y_history):
        # input_encoded: batch_size * T - 1 * encoder_hidden_size
        # y_history: batch_size * (T-1)
        # Initialize hidden and cell, 1 * batch_size * decoder_hidden_size
        hidden = self.init_hidden(input_encoded)
        cell = self.init_hidden(input_encoded)
        # hidden.requires_grad = False
        # cell.requires_grad = False
        for t in range(self.T - 1):
            # Eqn. 12-13: compute attention weights
            ## batch_size * T * (2*decoder_hidden_size + encoder_hidden_size)
            x = torch.cat((hidden.repeat(self.T - 1, 1, 1).permute(1, 0, 2),
                           cell.repeat(self.T - 1, 1, 1).permute(1, 0, 2), input_encoded), dim = 2)
            x = F.softmax(self.attn_layer(x.view(-1, 2 * self.decoder_hidden_size + self.encoder_hidden_size
                                                )).view(-1, self.T - 1)) # batch_size * T - 1, row sum up to 1
            # Eqn. 14: compute context vector
            context = torch.bmm(x.unsqueeze(1), input_encoded)[:, 0, :] # batch_size * encoder_hidden_size
            if t < self.T - 1:
                # Eqn. 15
                y_tilde = self.fc(torch.cat((context, y_history[:, t].unsqueeze(1)), dim = 1)) # batch_size * 1
                # Eqn. 16: LSTM
                self.lstm_layer.flatten_parameters()
                _, lstm_output = self.lstm_layer(y_tilde.unsqueeze(0), (hidden, cell))
                hidden = lstm_output[0] # 1 * batch_size * decoder_hidden_size
                cell = lstm_output[1] # 1 * batch_size * decoder_hidden_size
        # Eqn. 22: final output
        y_pred = self.fc_final(torch.cat((hidden[0], context), dim = 1))
        # self.logger.info("hidden %s context %s y_pred: %s", hidden[0][0][:10], context[0][:10], y_pred[:10])
        return y_pred

    def init_hidden(self, x):
        return Variable(x.data.new(1, x.size(0), self.decoder_hidden_size).zero_())


In [55]:
# Train the model
class da_rnn:
    def __init__(self, file_data, logger, encoder_hidden_size = 64, decoder_hidden_size = 64, T = 10,
                 learning_rate = 0.01, batch_size = 128, parallel = True, debug = False):
        self.T = T
        dat = pd.read_csv(file_data, nrows = 100 if debug else None)
        self.logger = logger
        self.logger.info("Shape of data: %s.\nMissing in data: %s.", dat.shape, dat.isnull().sum().sum())

        self.X = dat.loc[:, [x for x in dat.columns.tolist() if x != 'NDX']].as_matrix()
        self.y = np.array(dat.NDX)
        self.batch_size = batch_size

        self.encoder = encoder(input_size = self.X.shape[1], hidden_size = encoder_hidden_size, T = T,
                              logger = logger).cuda()
        self.decoder = decoder(encoder_hidden_size = encoder_hidden_size,
                               decoder_hidden_size = decoder_hidden_size,
                               T = T, logger = logger).cuda()

        if parallel:
            self.encoder = nn.DataParallel(self.encoder)
            self.decoder = nn.DataParallel(self.decoder)

        self.encoder_optimizer = optim.Adam(params = itertools.ifilter(lambda p: p.requires_grad, self.encoder.parameters()),
                                           lr = learning_rate)
        self.decoder_optimizer = optim.Adam(params = itertools.ifilter(lambda p: p.requires_grad, self.decoder.parameters()),
                                           lr = learning_rate)
        # self.learning_rate = learning_rate

        self.train_size = int(self.X.shape[0] * 0.7)
        self.y = self.y - np.mean(self.y[:self.train_size]) # Question: why Adam requires data to be normalized?
        self.logger.info("Training size: %d.", self.train_size)

    def train(self, n_epochs = 10):
        iter_per_epoch = int(np.ceil(self.train_size * 1. / self.batch_size))
        logger.info("Iterations per epoch: %3.3f ~ %d.", self.train_size * 1. / self.batch_size, iter_per_epoch)
        self.iter_losses = np.zeros(n_epochs * iter_per_epoch)
        self.epoch_losses = np.zeros(n_epochs)

        self.loss_func = nn.MSELoss()

        n_iter = 0

        learning_rate = 1.

        for i in range(n_epochs):
            perm_idx = np.random.permutation(self.train_size - self.T)
            j = 0
            while j < self.train_size:
                batch_idx = perm_idx[j:(j + self.batch_size)]
                X = np.zeros((len(batch_idx), self.T - 1, self.X.shape[1]))
                y_history = np.zeros((len(batch_idx), self.T - 1))
                y_target = self.y[batch_idx + self.T]

                for k in range(len(batch_idx)):
                    X[k, :, :] = self.X[batch_idx[k] : (batch_idx[k] + self.T - 1), :]
                    y_history[k, :] = self.y[batch_idx[k] : (batch_idx[k] + self.T - 1)]

                loss = self.train_iteration(X, y_history, y_target)
                self.iter_losses[i * iter_per_epoch + j / self.batch_size] = loss
                #if (j / self.batch_size) % 50 == 0:
                #    self.logger.info("Epoch %d, Batch %d: loss = %3.3f.", i, j / self.batch_size, loss)
                j += self.batch_size
                n_iter += 1

                if n_iter % 10000 == 0 and n_iter > 0:
                    for param_group in self.encoder_optimizer.param_groups:
                        param_group['lr'] = param_group['lr'] * 0.9
                    for param_group in self.decoder_optimizer.param_groups:
                        param_group['lr'] = param_group['lr'] * 0.9

            self.epoch_losses[i] = np.mean(self.iter_losses[range(i * iter_per_epoch, (i + 1) * iter_per_epoch)])
            if i % 10 == 0:
                self.logger.info("Epoch %d, loss: %3.3f.", i, self.epoch_losses[i])

            if i % 10 == 0:
                y_train_pred = self.predict(on_train = True)
                y_test_pred = self.predict(on_train = False)
                y_pred = np.concatenate((y_train_pred, y_test_pred))
                plt.figure()
                plt.plot(range(1, 1 + len(self.y)), self.y, label = "True")
                plt.plot(range(self.T , len(y_train_pred) + self.T), y_train_pred, label = 'Predicted - Train')
                plt.plot(range(self.T + len(y_train_pred) , len(self.y) + 1), y_test_pred, label = 'Predicted - Test')
                plt.legend(loc = 'upper left')
                plt.show()

    def train_iteration(self, X, y_history, y_target):
        self.encoder_optimizer.zero_grad()
        self.decoder_optimizer.zero_grad()

        input_weighted, input_encoded = self.encoder(Variable(torch.from_numpy(X).type(torch.FloatTensor).cuda()))
        y_pred = self.decoder(input_encoded, Variable(torch.from_numpy(y_history).type(torch.FloatTensor).cuda()))

        y_true = Variable(torch.from_numpy(y_target).type(torch.FloatTensor).cuda())
        loss = self.loss_func(y_pred, y_true)
        loss.backward()

        self.encoder_optimizer.step()
        self.decoder_optimizer.step()

        return loss.data[0]

    def predict(self, on_train = False):
        if on_train:
            y_pred = np.zeros(self.train_size - self.T + 1)
        else:
            y_pred = np.zeros(self.X.shape[0] - self.train_size)

        i = 0
        while i < len(y_pred):
            batch_idx = np.array(range(len(y_pred)))[i : (i + self.batch_size)]
            X = np.zeros((len(batch_idx), self.T - 1, self.X.shape[1]))
            y_history = np.zeros((len(batch_idx), self.T - 1))
            for j in range(len(batch_idx)):
                if on_train:
                    X[j, :, :] = self.X[range(batch_idx[j], batch_idx[j] + self.T - 1), :]
                    y_history[j, :] = self.y[range(batch_idx[j],  batch_idx[j]+ self.T - 1)]
                else:
                    X[j, :, :] = self.X[range(batch_idx[j] + self.train_size - self.T, batch_idx[j] + self.train_size - 1), :]
                    y_history[j, :] = self.y[range(batch_idx[j] + self.train_size - self.T,  batch_idx[j]+ self.train_size - 1)]

            y_history = Variable(torch.from_numpy(y_history).type(torch.FloatTensor).cuda())
            _, input_encoded = self.encoder(Variable(torch.from_numpy(X).type(torch.FloatTensor).cuda()))
            y_pred[i:(i + self.batch_size)] = self.decoder(input_encoded, y_history).cpu().data.numpy()[:, 0]
            i += self.batch_size
        return y_pred


In [61]:
io_dir = ':/'

model = da_rnn(file_data = '{}/Noise1/Noise.npy'.format(io_dir), logger = logger, parallel = False,
              learning_rate = .001)

model.train(n_epochs = 500)

y_pred = model.predict()

plt.figure()
plt.semilogy(range(len(model.iter_losses)), model.iter_losses)
plt.show()

plt.figure()
plt.semilogy(range(len(model.epoch_losses)), model.epoch_losses)
plt.show()

plt.figure()
plt.plot(y_pred, label = 'Predicted')
plt.plot(model.y[model.train_size:], label = "True")
plt.legend(loc = 'upper left')
plt.show()


FileNotFoundError: File b'{}/Noise1/Noise.npy' does not exist

In [13]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from torch.autograd import Variable
from torch.utils.data import Dataset

import os
import sys
import glob

import sklearn
import scipy
import librosa
import keras


import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from scipy.io import wavfile
import os
import sys

import sklearn
import scipy
import librosa
import keras

import numpy as np
from tqdm import tqdm

In [14]:
###Deprecated Code, here to be used for confirmation
class audioDataset(Dataset):
    """
    Instance of Dataset with additional code to create npy file with MFCCs should fix later.
    """
    
    
    def __init__(self, DATAPATH, 
                 train = True, 
                 labels = 'labels.txt', 
                 x_data = np.zeros(shape = (100,)),
                 y_data = np.zeros(shape = (100,))):
        self.DATAPATH = DATAPATH
        self.labels = labels
        #Code not needed once data has been npy'd
        self.get_train_test()
        self.get_labels()
        self.train = train
        self.x_data = x_data
        self.y_data = y_data
                
        
        
        
    def __getitem__(self, index):
        
        #train
        if self.train == True:
            self.X_train = np.load(self.DATAPATH + self.labels[0] + '_train' + '.npy') 

            self.y_train = np.full(self.X_train.shape[0], fill_value= (0 + 1))
            self.X_train = self.X_train.astype(float)
            self.y_train = self.y_train.astype(float)
            self.X_train = torch.Tensor(self.X_train)
            self.y_train = torch.Tensor(self.y_train)
            self.x_data = self.X_train
            self.y_data = self.y_train
        
        
        #test
        else:

            self.X_test = np.load(self.DATAPATH + self.labels[0] + '_test' + '.npy')
            self.y_test = np.full(self.X_test.shape[0], fill_value= (0 + 1))
            self.X_test = self.X_test.astype(float)
            self.y_test = self.y_test.astype(float)
            self.X_test = torch.Tensor(self.X_test)
            self.y_test = torch.Tensor(self.y_test)
            self.x_data = self.X_test
            self.y_data= self.y_test          
            
        sample = {'data' : self.x_data, 'labels': self.y_data}
        return sample
    
    
    
    def __len__(self):
        n = self.x_data.shape[0]
        return n
    
    def get_labels(self):
        self.labels = os.listdir(self.DATAPATH) 
        label_indices = np.arange(0, len(self.labels))
        return self.labels, label_indices, keras.utils.np_utils.to_categorical(label_indices)

    # Handy function to convert wav2mfcc
    def wav2mfcc(self, file_path, max_len=40):
        mfcc = []
        rate, _ = wavfile.read(file_path)

        wave, sr = librosa.load(file_path, mono=True, sr=rate)

        wavelength = len(wave)
        if wavelength < sr:
            print('pass') 
            pass

        else:
            mfcc = librosa.feature.mfcc(wave, sr=sr)

            # If maximum length exceeds mfcc lengths then pad the remaining ones
            if (max_len > mfcc.shape[1]):
                pad_width = max_len - mfcc.shape[1]
                mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')

            # Else cutoff the remaining parts
            else:
                mfcc = mfcc[:, :max_len]

        return mfcc


    def save_data_to_array(self, max_len=11):
        self.labels, _, _ = self.get_labels()

        for label in self.labels:
            # Init mfcc vectors
            mfcc_vectors = []
            print(label)
            
        
            wavfiles = [self.DATAPATH + label + '/' + wavfile for wavfile in os.listdir(self.DATAPATH + '/' + label)]
            
            exclude = set(['Noise.npy', 'Noise_train.npy', 'Noise_test.npy'])
            #allows exclusion of bad folders and modified files
            [wavfiles.remove(d) for d in list(wavfiles) if d in exclude]   
            
            
            for wavfile in tqdm(wavfiles, "Saving vectors of label - '{}'".format(label)):
                
                
                
                mfcc = self.wav2mfcc(wavfile, max_len=max_len)
                if len(mfcc) > 0:
                    mfcc_vectors.append(mfcc)
            np.save(label + '.npy', mfcc_vectors)
        return mfcc_vectors 

    def build_npy():
        save_data_to_array(max_len=11)
        
    def get_train_test(self, split_ratio=0.75, random_state=42):
        # Get available labels
        self.labels, indices, _ = self.get_labels()

        # Getting first arrays
        for file in glob.glob(self.DATAPATH):
            if file.endswith('.npy'):
                print(files)
                X = np.load(files)
                y = np.full(X.shape[0], fill_value= (0 + 1), dtype = int)
       

                assert X.shape[0] == len(y)
                X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size= (1 - split_ratio), random_state=random_state)
        
                np.save(self.DATAPATH + self.labels[0] + '_train', X_train)
                np.save(self.DATAPATH + self.labels[0] + '_test', X_test)
        return 0
        


In [365]:
#Linux Call

train_dataset = audioDataset(DATAPATH = '/media/tim/0ACD-1A94/Noise1/', train=True)

test_dataset = audioDataset(DATAPATH = '/media/tim/0ACD-1A94/Noise1/', train=False)

Noise


Saving vectors of label - 'Noise':   6%|▌         | 507/8824 [03:55<1:02:30,  2.22it/s]

pass


Saving vectors of label - 'Noise':   6%|▌         | 508/8824 [03:55<1:01:52,  2.24it/s]

pass


...

Saving vectors of label - 'Noise': 100%|█████████▉| 8823/8824 [1:06:45<00:00,  2.27it/s]

pass


Saving vectors of label - 'Noise': 100%|██████████| 8824/8824 [1:06:45<00:00,  2.30it/s]

pass
Noise.npy


NotADirectoryError: [Errno 20] Not a directory: '/media/tim/0ACD-1A94/Noise1//Noise.npy'

### 

In [15]:
#Whale dataset
train_dataset = audioDataset(DATAPATH = 'D:\Masters\Test/', train=True)

test_dataset = audioDataset(DATAPATH = 'D:\Masters\Test/', train=False)

In [18]:
batch_size = 20

In [19]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)



In [21]:
total_step = 1
data_iter = iter(train_loader)
iter_per_epoch = len(train_loader)
# Start training
for step in range(total_step): 
    if (step+1) % iter_per_epoch == 0:
        data_iter = iter(train_loader)

        
    # Fetch the images and labels and convert them to variables
    data, labels = next(data_iter)
    print((data[1]))


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Masters\\Test/blue001-v2_train.npy'

In [354]:
class encode(nn.Module):
    def __init__(self):
        super(encode, self).__init__()

        #self.fc1 = nn.Linear(4400, 400)
        #self.fc21 = nn.Linear(400, 20)
        #self.fc22 = nn.Linear(400, 20)
        #self.fc3 = nn.Linear(20, 400)
        #self.fc4 = nn.Linear(400, 4400)
        
        self.fc1 = nn.Linear(11, 10)
        self.fc2 = nn.Linear(10, 10)
        self.fc3gauss = nn.Linear(10, 5)

        
        

    def forward(self, data):
        #h1 = F.relu(self.fc1(x))
        #return self.fc21(h1), self.fc22(h1)

        x = F.dropout(self.fc1(data), p=0.25, training=self.training)
        x = F.relu(x)
        x = F.dropout(self.fc2(x), p=0.25, training=self.training)
        x = F.relu(x)
        xgauss = self.fc3gauss(x)
        return xgauss
        
class decode(nn.Module):
    def __init__(self):
        super(decode, self).__init__()
        self.fc4 = nn.Linear(5, 10)
        self.fc5 = nn.Linear(10, 10)
        self.fc6 = nn.Linear(10, 11)
        
    def forward(self, data):
        #h3 = F.relu(self.fc3(z))
        #return F.sigmoid(self.fc4(h3))
        x = self.fc4(data)
        x = F.dropout(x, p=0.25, training=self.training)
        x = F.relu(x)
        x = self.fc5(x)
        x = F.dropout(x, p=0.25, training=self.training)
        x = self.fc6(x)
        return F.sigmoid(x)



In [351]:
def to_var(x):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x)    


# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x,, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 4400), size_average=False)

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD

In [355]:
Q = encode().cuda()
P = decode().cuda()

gen_lr = 0.0001

#encode/decode optimizers
optim_P = torch.optim.Adam(P.parameters(), lr=gen_lr)
optim_Q = torch.optim.Adam(Q.parameters(), lr=gen_lr)





In [369]:
EPS = 1e-15
z_red_dims = 120

data_iter = iter(train_loader)
iter_per_epoch = len(train_loader)
total_step = 50000

# Start training
for step in range(total_step):

    # Reset the data_iter
    if (step+1) % iter_per_epoch == 0:
        data_iter = iter(train_loader)

    # Fetch the images and labels and convert them to variables
    data, labels = next(data_iter)
    data, labels = to_var(data), to_var(labels)


    #reconstruction loss
    Q.zero_grad()
    P.zero_grad()


    z_sample = Q(data)   #encode to z
    X_sample = P(z_sample) #decode to X reconstruction

    
    recon_loss = F.mse_loss(X_sample+EPS,data+EPS)

    recon_loss.backward()
    optim_P.step()
    optim_Q.step()

    
    if (step+1) % 100 == 0:
         print ('Step [%d/%d], Loss: %.4f' 
              %(step+1, total_step, recon_loss.data[0]))


/home/tim/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:38: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Step [100/50000], Loss: 1489.6381
Step [200/50000], Loss: 1522.9851
Step [300/50000], Loss: 948.4675
Step [400/50000], Loss: 1566.0013
Step [500/50000], Loss: 1476.9844
Step [600/50000], Loss: 1011.3157
Step [700/50000], Loss: 1831.7657
Step [800/50000], Loss: 1949.4116
Step [900/50000], Loss: 1987.8362
Step [1000/50000], Loss: 616.6523
Step [1100/50000], Loss: 1814.9812
Step [1200/50000], Loss: 1581.8027
Step [1300/50000], Loss: 1433.5880
Step [1400/50000], Loss: 1446.0918
Step [1500/50000], Loss: 1312.1271
Step [1600/50000], Loss: 1949.7068
Step [1700/50000], Loss: 1780.7463
Step [1800/50000], Loss: 2524.9617
Step [1900/50000], Loss: 1607.1373
Step [2000/50000], Loss: 1616.3331
Step [2100/50000], Loss: 1160.8422
Step [2200/50000], Loss: 1548.9543
Step [2300/50000], Loss: 1450.1233
Step [2400/50000], Loss: 1574.4625
Step [2500/50000], Loss: 1747.0778
Step [2600/50000], Loss: 1205.8557
Step [2700/50000], Loss: 819.3865
Step [2800/50000], Loss: 1882.4668
Step [2900/50000], Loss: 1885.50

KeyboardInterrupt: 

## 
epochs=10
seed=1
log_interval = 10


def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = Variable(data)

        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, logvar = model(data)
        loss = loss_function(recon_batch, data, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))


def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = Variable(data)
            
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(batch_size, 1, 28, 28)[:n]])
                save_image(comparison.cpu(),
                         'results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))


for epoch in range(1, epochs + 1):
    train(epoch)
    test(epoch)
    with torch.no_grad():
        sample = torch.randn(64, 20).to(device)
        sample = model.decode(sample).cpu()
        save_image(sample.view(64, 1, 28, 28),
                   'results/sample_' + str(epoch) + '.png')


### train model ( work on)

    def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
        since = time.time()

        best_model_wts = copy.deepcopy(model.state_dict())
        best_acc = 0.0

        for epoch in range(num_epochs):
            print('Epoch {}/{}'.format(epoch, num_epochs - 1))
            print('-' * 10)

            # Each epoch has a training and validation phase
            for phase in ['train', 'val']:
                if phase == 'train':
                    scheduler.step()
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in dataloaders[phase]:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)

                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects.double() / dataset_sizes[phase]

                print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                    phase, epoch_loss, epoch_acc))

                # deep copy the model
                if phase == 'val' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())

            print()

        time_elapsed = time.time() - since
        print('Training complete in {:.0f}m {:.0f}s'.format(
            time_elapsed // 60, time_elapsed % 60))
        print('Best val Acc: {:4f}'.format(best_acc))

        # load best model weights
        model.load_state_dict(best_model_wts)
        return model

In [327]:
DATA_PATH = '/media/tim/0ACD-1A94/Noise1/'

In [335]:
X_train4 = np.load(DATA_PATH +  'Noise_train' + '.npy')

In [329]:
import pandas as pd

In [340]:
X_train4.shape

(3981, 20, 11)

In [334]:
df = pd.DataFrame(X_train4)




ValueError: Must pass 2-d input

In [333]:
df

""


In [10]:
import torch

## Pytorch Audio dataloader

The pytorch audioloader has the functionality to implement a couple of transforms. The program includes mel spectrograms. However it may be capable of implementing the code as raw. Work is needed on this.


### Implementation


In [9]:
data, samplerate = torch.torchaudio.load('/media/tim/0ACD-1A94/Noise1/' )

NameError: name 'torchaudio' is not defined

In [2]:
import pandas as pd

In [4]:
df = pd.DataFrame()

In [74]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from torch.autograd import Variable
from torch.utils.data import Dataset
#from torchaudio import transforms
#from torchaudio import Datasets
import os
import sys
from glob import glob
import datetime
import re

import sklearn
import scipy
import librosa
import keras
import matplotlib.pyplot as plt


import numpy as np
from tqdm import tqdm

In [156]:

class audioDataset(Dataset):
    """
    Instance of Dataset with additional code to create npy file with MFCCs should fix later.
    """
    
    
    def __init__(self,  DATAPATH, 
                 train = True, 
                 labels = 'labels.txt',
                 audio_files= 'labels.txt',
                 x_data = np.zeros(shape = (100,)),
                 y_data = np.zeros(shape = (100,)),
                 file = 'test1'):
        self.DATAPATH = DATAPATH
        self.labels = labels
        self.audio_files = audio_files
        self.file = file
        self.train = train
        self.x_data = x_data
        self.y_data = y_data
        #Code not needed once data has been npy'd
        self.get_train_test()
   
    def __getitem__(self, index):
        
        #train
        if self.train == True:
            print('train')
            self.train_data = np.load(self.DATAPATH + 'testing' + '_train' + '.npy') 
            print(self.train_data)
            #self.y_train = np.full(self.X_train.shape[0], fill_value= (0 + 1))
            ##self.train_data = self.train_data.astype(float)
            #self.y_train = self.y_train.astype(float)
            #self.X_train = torch.Tensor(self.X_train)
            #self.y_train = torch.Tensor(self.y_train)
            #self.x_data = self.X_train
            #self.y_data = self.y_train
            ##self.x_data = torch.tensor(self.train_data[:-2])
            ##self.y_data = torch.tensor(self.train_data[-1])

            
        
        #test
        else:

            #self.X_test = np.load(self.DATAPATH + self.labels[0].replace('.npy', '')  + '_test' + '.npy')
            #self.y_test = np.full(self.X_test.shape[0], fill_value= (0 + 1))
            
            #self.X_test = self.X_test.astype(float)
            #self.y_test = self.y_test.astype(float)
            #self.X_test = torch.Tensor(self.X_test)
            #self.y_test = torch.Tensor(self.y_test)
            #self.x_data = self.X_test
            #self.y_data= self.y_test 
            self.test_data = np.load(self.DATAPATH + 'testing' + '_test' + '.npy') 
            print(self.test_data)
            self.train_data = self.train_data.astype(float)
            self.x_data = torch.tensor(self.test_data[:-2])
            self.y_data = torch.tensor(self.test_data[-1])
            
        sample = {'data' : self.x_data, 'labels': self.y_data}
        print('sample', sample)
        return sample
    
    
    
    def __len__(self):
        #Reads in dimension 1 of the 3-D x_data array(width of dataset, length of MFCC, MFCC amount)
        n = self.x_data.shape[0]
        return n
    
    def get_labels(self):
        """
        get_labels: Reads in the whale audio files from the DATAPATH folder and 
        collects all the audio files that have been processed. This checks for 
        the .npy file extension but discludes the _train and _test appendings to the file
        Usage:
        self.labels - For collecting class labels for the classifier.
        self.audio_labels - For creating the MFCC npy files from the audio data.
        
            
        """
        included_extensions = '.npy'
        excluded_extensions = set(['_train.npy', '_test.npy'])
        excluded_extensions_audio = set({'.npy'})
            #allows exclusion of bad folders and modified files
          
        self.labels = [fn for fn in os.listdir(self.DATAPATH)
                      if any(fn.endswith(ext) for ext in included_extensions)]
        self.labels = [fn for fn in self.labels
                      if not any(fn.endswith(exclude) for exclude in excluded_extensions)]
        
        self.audio_files = os.listdir(self.DATAPATH) 
        self.audio_files = [fn for fn in self.audio_files if not any(fn.endswith(exclude) for exclude in excluded_extensions_audio)]
        
        
        
        
        self.label_indices = np.arange(0, len(self.labels))
        self.categorical_class = keras.utils.np_utils.to_categorical(self.label_indices)

        

    # Handy function to convert wav2mfcc
    def wav2mfcc(self, file_path, max_len=40):
        mfcc = []
        rate, _ = wavfile.read(file_path)

        wave, sr = librosa.load(file_path, mono=True, sr=rate)

        wavelength = len(wave)
        if wavelength < sr:
            print('pass') 
            pass

        else:
            mfcc = librosa.feature.mfcc(wave, sr=sr)

            # If maximum length exceeds mfcc lengths then pad the remaining ones
            if (max_len > mfcc.shape[1]):
                pad_width = max_len - mfcc.shape[1]
                mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')

            # Else cutoff the remaining parts
            else:
                mfcc = mfcc[:, :max_len]

        return mfcc


    def save_data_to_array(self, max_len=11):
        
        self.get_labels()
        array_stack = []

        for label in self.audio_labels:
            # Init mfcc vectors
            
            
        
            wavfiles = [self.DATAPATH + label + '/' + wavfile for wavfile in os.listdir(self.DATAPATH + '/' + label)]
            
            exclude = set(['Noise.npy', 'Noise_train.npy', 'Noise_test.npy'])
            #allows exclusion of bad folders and modified files
            [wavfiles.remove(d) for d in list(wavfiles) if d in exclude]   
            
            for wavfile in tqdm(wavfiles, "Saving vectors of label - '{}'".format(label)):
                mfcc = self.wav2mfcc(wavfile, max_len=max_len)
                if len(mfcc) > 0:
                    mfcc_vectors.append(mfcc)
                    
            
            np.save(label + '.npy', mfcc_vectors)
            
        return mfcc_vectors 
    
    
    

    def build_npy():
        save_data_to_array(max_len=11)
        
    def get_train_test(self, split_ratio=0.75, random_state=42):
        # Get available labels
        self.get_labels()
        X = np.load(self.DATAPATH + self.labels[0])
        print(X.shape)
        
        y = np.zeros(len(X))
        
        # Getting first arrays
            
            # Append all of the dataset into one single array, same goes for y
        for i, label in enumerate(self.labels[1:]):
            path = self.DATAPATH + label
            label = re.sub('\.npy$', '', path)
            x = np.load(label + '.npy')
            X = np.vstack((X, x))
            y1 = np.ones(len(x))
            vals = self.label_indices[i+1]
            mm = vals * y1
            y=scipy.sparse.hstack((y,mm))
            
        
###### HOW CAN I CONVERT AN ARRAY INTO 2D


            #y1 = y1*self.label_indices[i+1]
            #y = np.vstack((y, y1))
            
            #print(y)
            #for file in files: 
            #    if file.endswith('.npy'):
            #        print(file)
            #        X = np.load(current_path+file)
            #        y = 

        y = y.T
        X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size= (1 - split_ratio), random_state=random_state)
        training_set = (X_train, y_train)
        print(training_set[:-1])
        test_set = (X_test, y_test)
        
        
        np.save((self.DATAPATH + 'testing'+ '_train'), training_set)
        np.save((self.DATAPATH + 'testing' + '_test'), test_set)
        #print(y)
        return 
        


In [157]:
#Whale dataset
train_dataset = audioDataset(DATAPATH = 'D:\Masters\Test/', train=True, file = 'test1')

test_dataset = audioDataset(DATAPATH = 'D:\Masters\Test/', train=False, file = 'test1')

(486, 20, 11)
(array([[[  3.34265136e+01,   0.00000000e+00,   0.00000000e+00, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [  4.31035538e+00,   0.00000000e+00,   0.00000000e+00, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [ -8.25402039e+01,   0.00000000e+00,   0.00000000e+00, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        ..., 
        [ -2.78412638e-02,   0.00000000e+00,   0.00000000e+00, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [ -1.12882735e+01,   0.00000000e+00,   0.00000000e+00, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [ -5.24217569e+00,   0.00000000e+00,   0.00000000e+00, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00]],

       [[  7.44204899e+01,   0.00000000e+00,   0.00000000e+00, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [ -1.34233705e+01,   0.00000000e+00, 

           0.00000000e+00,   0.00000000e+00,   0.00000000e+00]]]),)


In [144]:
batch_size = 20

In [145]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)


In [146]:
train_dataset.labels


['blue001-v2.npy',
 'fin-01.npy',
 'minke-01.npy',
 'SouthernRightWhale001-v1.npy']

In [147]:
for i in range(len(train_dataset)):
    sample = train_dataset[i]

    print(i, sample['labels'].size())

    if i == 3:
        break

train
[ array([[[  3.34265136e+01,   0.00000000e+00,   0.00000000e+00, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [  4.31035538e+00,   0.00000000e+00,   0.00000000e+00, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [ -8.25402039e+01,   0.00000000e+00,   0.00000000e+00, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        ..., 
        [ -2.78412638e-02,   0.00000000e+00,   0.00000000e+00, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [ -1.12882735e+01,   0.00000000e+00,   0.00000000e+00, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [ -5.24217569e+00,   0.00000000e+00,   0.00000000e+00, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00]],

       [[  7.44204899e+01,   0.00000000e+00,   0.00000000e+00, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [ -1.34233705e+01,   0.00000000e+00,   0.000

TypeError: 'int' object is not callable

In [113]:
n_categories = 4
n_hidden = 128
input_size = 50
num_epochs = 100000

In [114]:


class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

rnn = RNN(input_size, n_hidden, n_categories)

In [115]:
learning_rate = 0.005 # If you set this too high, it might explode. If too low, it might not learn

def train(category_tensor, line_tensor):
    hidden = rnn.initHidden()

    rnn.zero_grad()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    loss = criterion(output, category_tensor)
    loss.backward()

    # Add parameters' gradients to their values, multiplied by learning rate
    for p in rnn.parameters():
        p.data.add_(-learning_rate, p.grad.data)

    return output, loss.item()

In [116]:
def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    return all_categories[category_i], category_i


In [117]:
import time
import math

n_iters = 100000
print_every = 5000
plot_every = 1000



# Keep track of losses for plotting
current_loss = 0
all_losses = []

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()


        

In [119]:
for epoch in range(num_epochs):
    for i, (data, labels) in enumerate(train_loader):
        # Load images as Variable
        #######################
        #  USE GPU FOR MODEL  #
        #######################
        if torch.cuda.is_available():
            audio = Variable(data.cuda())
            labels = Variable(labels.cuda())
        else:
            audio = Variable(data)
            labels = Variable(labels)
            
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
        
        # Forward pass to get output/logits
        # outputs.size() --> 100, 10
        outputs = rnn(audio, n_hidden, n_categories)
        
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)
        
        # Getting gradients w.r.t. parameters
        loss.backward()
        
        # Updating parameters
        optimizer.step()
        
        iter += 1
        
        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for data, labels in test_loader:
                #######################
                #  USE GPU FOR MODEL  #
                #######################
                if torch.cuda.is_available():
                    audio = Variable(data.cuda())
                else:
                    audio = Variable(data)
                
                # Forward pass only to get logits/output
                outputs = rnn(audio, n_hidden, n_categories)
                
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                
                # Total number of labels
                total += labels.size(0)
                
                # Total correct predictions
                #######################
                #  USE GPU FOR MODEL  #
                #######################
                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum()
                else:
                    correct += (predicted == labels).sum()
            
            accuracy = 100 * correct / total
            
            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.data[0], accuracy))

train
[[[  3.34265136e+01   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [  4.31035538e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [ -8.25402039e+01   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  ..., 
  [ -2.78412638e-02   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [ -1.12882735e+01   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [ -5.24217569e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]]

 [[  7.44204899e+01   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [ -1.34233705e+01   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [ -1.07276609e+02   0.00000000e+00   0.00000000e+00 ...,   0.00000000e

     0.00000000e+00   0.00000000e+00]]]
train
[[[  3.34265136e+01   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [  4.31035538e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [ -8.25402039e+01   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  ..., 
  [ -2.78412638e-02   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [ -1.12882735e+01   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [ -5.24217569e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]]

 [[  7.44204899e+01   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [ -1.34233705e+01   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [ -1.07276609e+02   0.00000000

     0.00000000e+00   0.00000000e+00]]]
train
[[[  3.34265136e+01   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [  4.31035538e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [ -8.25402039e+01   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  ..., 
  [ -2.78412638e-02   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [ -1.12882735e+01   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [ -5.24217569e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]]

 [[  7.44204899e+01   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [ -1.34233705e+01   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [ -1.07276609e+02   0.00000000

     0.00000000e+00   0.00000000e+00]]]
train
[[[  3.34265136e+01   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [  4.31035538e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [ -8.25402039e+01   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  ..., 
  [ -2.78412638e-02   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [ -1.12882735e+01   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [ -5.24217569e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]]

 [[  7.44204899e+01   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [ -1.34233705e+01   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [ -1.07276609e+02   0.00000000

     0.00000000e+00   0.00000000e+00]]]
train
[[[  3.34265136e+01   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [  4.31035538e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [ -8.25402039e+01   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  ..., 
  [ -2.78412638e-02   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [ -1.12882735e+01   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [ -5.24217569e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]]

 [[  7.44204899e+01   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [ -1.34233705e+01   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [ -1.07276609e+02   0.00000000

...

     0.00000000e+00   0.00000000e+00]]]
train
[[[  3.34265136e+01   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [  4.31035538e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [ -8.25402039e+01   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  ..., 
  [ -2.78412638e-02   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [ -1.12882735e+01   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [ -5.24217569e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]]

 [[  7.44204899e+01   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [ -1.34233705e+01   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [ -1.07276609e+02   0.00000000

     0.00000000e+00   0.00000000e+00]]]
train
[[[  3.34265136e+01   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [  4.31035538e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [ -8.25402039e+01   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  ..., 
  [ -2.78412638e-02   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [ -1.12882735e+01   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [ -5.24217569e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]]

 [[  7.44204899e+01   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [ -1.34233705e+01   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [ -1.07276609e+02   0.00000000

     0.00000000e+00   0.00000000e+00]]]
train
[[[  3.34265136e+01   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [  4.31035538e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [ -8.25402039e+01   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  ..., 
  [ -2.78412638e-02   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [ -1.12882735e+01   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [ -5.24217569e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]]

 [[  7.44204899e+01   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [ -1.34233705e+01   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [ -1.07276609e+02   0.00000000

     0.00000000e+00   0.00000000e+00]]]


TypeError: Variable data has to be a tensor, but got str

In [ ]:



class data_creator():
        
        
        
    def init:(self,  DATAPATH, 
                 train = True, 
                 labels = 'labels.txt',
                 audio_files= 'labels.txt',
                 x_data = np.zeros(shape = (100,)),
                 y_data = np.zeros(shape = (100,)),
                 file = 'test1')
        self.DATAPATH = DATAPATH
        self.labels = labels
        self.audio_files = audio_files
        self.file = file
        self.train = train
        self.x_data = x_data
        self.y_data = y_data
        #Code not needed once data has been npy'd
        self.get_train_test()):
    
    def get_train_test(self, split_ratio=0.75, random_state=42):
        # Get available labels
        self.get_labels()
        X = np.load(self.DATAPATH + self.labels[0])
        print(X.shape)
        
        y = np.zeros(len(X))
        
        # Getting first arrays
            
            # Append all of the dataset into one single array, same goes for y
        for i, label in enumerate(self.labels[1:]):
            path = self.DATAPATH + label
            label = re.sub('\.npy$', '', path)
            x = np.load(label + '.npy')
            X = np.vstack((X, x))
            y1 = np.ones(len(x))
            vals = self.label_indices[i+1]
            mm = vals * y1
            y=scipy.sparse.hstack((y,mm))
            
        
###### HOW CAN I CONVERT AN ARRAY INTO 2D


            #y1 = y1*self.label_indices[i+1]
            #y = np.vstack((y, y1))
            
            #print(y)
            #for file in files: 
            #    if file.endswith('.npy'):
            #        print(file)
            #        X = np.load(current_path+file)
            #        y = 

        y = y.T
        X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size= (1 - split_ratio), random_state=random_state)
        training_set = (X_train, y_train)
        print(training_set[:-1])
        test_set = (X_test, y_test)
        
        
        np.save((self.DATAPATH + 'testing'+ '_train'), training_set)
        np.save((self.DATAPATH + 'testing' + '_test'), test_set)
        #print(y)
        return 



    def get_labels(self):
        """
        get_labels: Reads in the whale audio files from the DATAPATH folder and 
        collects all the audio files that have been processed. This checks for 
        the .npy file extension but discludes the _train and _test appendings to the file
        Usage:
        self.labels - For collecting class labels for the classifier.
        self.audio_labels - For creating the MFCC npy files from the audio data.
        
            
        """
        included_extensions = '.npy'
        excluded_extensions = set(['_train.npy', '_test.npy'])
        excluded_extensions_audio = set({'.npy'})
            #allows exclusion of bad folders and modified files
          
        self.labels = [fn for fn in os.listdir(self.DATAPATH)
                      if any(fn.endswith(ext) for ext in included_extensions)]
        self.labels = [fn for fn in self.labels
                      if not any(fn.endswith(exclude) for exclude in excluded_extensions)]
        
        self.audio_files = os.listdir(self.DATAPATH) 
        self.audio_files = [fn for fn in self.audio_files if not any(fn.endswith(exclude) for exclude in excluded_extensions_audio)]
        
        
        
        
        self.label_indices = np.arange(0, len(self.labels))
        self.categorical_class = keras.utils.np_utils.to_categorical(self.label_indices)

        

    # Handy function to convert wav2mfcc
    def wav2mfcc(self, file_path, max_len=40):
        mfcc = []
        rate, _ = wavfile.read(file_path)

        wave, sr = librosa.load(file_path, mono=True, sr=rate)

        wavelength = len(wave)
        if wavelength < sr:
            print('pass') 
            pass

        else:
            mfcc = librosa.feature.mfcc(wave, sr=sr)

            # If maximum length exceeds mfcc lengths then pad the remaining ones
            if (max_len > mfcc.shape[1]):
                pad_width = max_len - mfcc.shape[1]
                mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')

            # Else cutoff the remaining parts
            else:
                mfcc = mfcc[:, :max_len]

        return mfcc


    def save_data_to_array(self, max_len=11):
        
        self.get_labels()
        array_stack = []

        for label in self.audio_labels:
            # Init mfcc vectorsself,  DATAPATH, 
                 train = True, 
                 labels = 'labels.txt',
                 audio_files= 'labels.txt',
                 x_data = np.zeros(shape = (100,)),
                 y_data = np.zeros(shape = (100,)),
                 file = 'test1'):
        self.DATAPATH = DATAPATH
        self.labels = labels
        self.audio_files = audio_files
        self.file = file
        self.train = train
        self.x_data = x_data
        self.y_data = y_data
        #Code not needed once data has been npy'd
        self.get_train_test()

            wavfiles = [self.DATAPATH + label + '/' + wavfile for wavfile in os.listdir(self.DATAPATH + '/' + label)]
            
            exclude = set(['Noise.npy', 'Noise_train.npy', 'Noise_test.npy'])
            #allows exclusion of bad folders and modified files
            [wavfiles.remove(d) for d in list(wavfiles) if d in exclude]   
            
            for wavfile in tqdm(wavfiles, "Saving vectors of label - '{}'".format(label)):
                mfcc = self.wav2mfcc(wavfile, max_len=max_len)
                if len(mfcc) > 0:
                    mfcc_vectors.append(mfcc)
                    
            
            np.save(label + '.npy', mfcc_vectors)
            
        return mfcc_vectors 
    
    
    

    def build_npy():
        save_data_to_array(max_len=11)
        
   




In [158]:

class audioDataset(Dataset):
    """
    Instance of Dataset with additional code to create npy file with MFCCs should fix later.
    """
    
    
    def __init__(self,  DATAPATH, 
                 train = True, 
                 labels = 'labels.txt',
                 audio_files= 'labels.txt',
                 x_data = np.zeros(shape = (100,)),
                 y_data = np.zeros(shape = (100,)),
                 file = 'test1'):
        self.DATAPATH = DATAPATH
        self.labels = labels
        self.audio_files = audio_files
        self.file = file
        self.train = train
        self.x_data = x_data
        self.y_data = y_data
        #Code not needed once data has been npy'd
        self.get_train_test()
    
        if self.train == True:
            xy = np.load(self.DATAPATH + 'testing' + '_train' + '.npy')
            
        else:
            xy = np.load(self.DATAPATH + 'testing' + '_test' + '.npy') 
        
        self.len = xy.shape[0]
        self.x_data = torch.from_numpy(xy[:, 0:-1])
        self.y_data = torch.from_numpy(xy[:, [-1]])
        
    def __getitem__(self, index):
    
        return self.x_data[index], self.y_data[index]
    
    
    
    def __len__(self):
        #Reads in dimension 1 of the 3-D x_data array(width of dataset, length of MFCC, MFCC amount)
       
        return self.len
    

        
